In [4]:
conda install -c conda-forge spacy

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\User\anaconda3

  added / updated specs:
    - spacy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aiohttp-3.7.4              |   py38h294d835_0         596 KB  conda-forge
    async-timeout-3.0.1        |          py_1000          11 KB  conda-forge
    boto3-1.17.56              |     pyhd8ed1ab_0          70 KB  conda-forge
    botocore-1.20.56           |     pyhd8ed1ab_0         4.6 MB  conda-forge
    bz2file-0.98               |             py_0           9 KB  conda-forge
    cachetools-4.2.1           |     pyhd8ed1ab_0          13 KB  conda-forge
    catalogue-2.0.



==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda






catalogue-2.0.3      | 31 KB     |            |   0% 
catalogue-2.0.3      | 31 KB     | #####2     |  52% 
catalogue-2.0.3      | 31 KB     | ########## | 100% 

smart_open-2.2.1     | 78 KB     |            |   0% 
smart_open-2.2.1     | 78 KB     | ##         |  20% 
smart_open-2.2.1     | 78 KB     | ########## | 100% 

google-crc32c-1.1.2  | 27 KB     |            |   0% 
google-crc32c-1.1.2  | 27 KB     | #####9     |  59% 
google-crc32c-1.1.2  | 27 KB     | ########## | 100% 

yarl-1.5.1           | 136 KB    |            |   0% 
yarl-1.5.1           | 136 KB    | #1         |  12% 
yarl-1.5.1           | 136 KB    | ########## | 100% 
yarl-1.5.1           | 136 KB    | ########## | 100% 

srsly-2.4.1          | 517 KB    |            |   0% 
srsly-2.4.1          | 517 KB    | 3          |   3% 
srsly-2.4.1          | 517 KB    | ##7        |  28% 
srsly-2.4.1          | 517 KB    | ########6  |  87% 
srsly-2.4.1          | 517 KB    | ########## | 100% 

async-timeout-3.0.1  

In [1]:
#import libraries required

import numpy as np, pandas as pd
import json
import ast 
from textblob import TextBlob
import nltk
import torch
import pickle
from scipy import spatial
import warnings
warnings.filterwarnings('ignore')
import spacy
from nltk import Tree
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [4]:
train = pd.read_csv("train.csv")

In [5]:
train.shape

(87599, 4)

### Loading Embedding dictionary

In [6]:
# opening sentence embeddings
with open("dict_embeddings1.pickle", "rb") as f:
    d1 = pickle.load(f)

In [7]:
with open("dict_embeddings2.pickle", "rb") as f:
    d2 = pickle.load(f)

In [8]:
# adding to same dict
dict_emb = dict(d1)
dict_emb.update(d2)

In [9]:
len(dict_emb)

60130

In [10]:
del d1, d2

## Data Processing

In [11]:
# if ans is in that sentence return sent index
def get_target(x):
    idx = -1
    for i in range(len(x["sentences"])):
        if x["text"] in x["sentences"][i]: 
            idx = i
    return idx

In [12]:
train.head(3)

,Context,Question,Answer Start Index,Answer Text
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building


In [13]:
train.shape

(87599, 4)

In [14]:
# remove rows, cols with missing values
train.dropna(inplace=True)

In [15]:
train.shape

(87598, 4)

In [16]:
type(train)

pandas.core.frame.DataFrame

In [17]:
col_names = train.columns.values

In [18]:
col_names

array(['Context', 'Question', 'Answer Start Index', 'Answer Text'],
      dtype=object)

In [20]:
train.columns = ['context', 'question', 'answer_start', 'text']

In [21]:
train.columns.values

array(['context', 'question', 'answer_start', 'text'], dtype=object)

In [22]:
def process_data(train):
    
    print("step 1")
    train['sentences'] = train['context'].apply(lambda x: [item.raw for item in TextBlob(x).sentences])
    
    print("step 2")
    train['target'] = train.apply(get_target, axis = 1)
    
    print("step 3")
    train['sent_emb'] = train['sentences'].apply(lambda x: [dict_emb[item][0] if item in\
                                                           dict_emb else np.zeros(4096) for item in x])
    print("step 4")
    train['quest_emb'] = train['question'].apply(lambda x: dict_emb[x] if x in dict_emb else np.zeros(4096) )
        
    return train   

In [23]:
train = process_data(train)

step 1
step 2
step 3
step 4


In [24]:
train.head(3)

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...",5,"[[0.055199962, 0.05013141, 0.04787038, 0.01624...","[[0.110100806, 0.11422941, 0.11560897, 0.05489..."
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...",2,"[[0.055199962, 0.05013141, 0.04787038, 0.01624...","[[0.10951651, 0.11030623, 0.052100062, 0.03053..."
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"[Architecturally, the school has a Catholic ch...",3,"[[0.055199962, 0.05013141, 0.04787038, 0.01624...","[[0.011956477, 0.14930709, 0.026600504, 0.0527..."


## Predicted Index based on Cosine Similarity & Euclidean Distance

In [25]:
# finding spatial dist b/w emb using cosine similarity
def cosine_sim(x):
    li = []
    for item in x["sent_emb"]:
        li.append(spatial.distance.cosine(item,x["quest_emb"][0]))
    return li   

In [26]:
# return min dist index
def pred_idx(distances):
    return np.argmin(distances)   

In [31]:
len(train)

87598

In [33]:
# making predictions for first 100 observations
train_1 = train.head(100)

In [34]:
# based on cosine similarity and euclidean dt, find the 
# sentence close to the question and return its index
def predictions(train):
    
    train["cosine_sim"] = train.apply(cosine_sim, axis = 1)
    train["diff"] = (train["quest_emb"] - train["sent_emb"])**2
    train["euclidean_dis"] = train["diff"].apply(lambda x: list(np.sum(x, axis = 1)))
    del train["diff"]
    
    print("cosine start")
    
    train["pred_idx_cos"] = train["cosine_sim"].apply(lambda x: pred_idx(x))
    train["pred_idx_euc"] = train["euclidean_dis"].apply(lambda x: pred_idx(x))
    
    return train
    

In [35]:
predicted = predictions(train_1)

cosine start


In [36]:
predicted.head(3)

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...",5,"[[0.055199962, 0.05013141, 0.04787038, 0.01624...","[[0.110100806, 0.11422941, 0.11560897, 0.05489...","[0.42473626136779785, 0.364050030708313, 0.347...","[14.563858, 15.262213, 17.398178, 14.272491, 1...",5,5
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...",2,"[[0.055199962, 0.05013141, 0.04787038, 0.01624...","[[0.10951651, 0.11030623, 0.052100062, 0.03053...","[0.4540746212005615, 0.32262009382247925, 0.35...","[12.889506, 12.285219, 16.843704, 8.361172, 11...",3,3
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"[Architecturally, the school has a Catholic ch...",3,"[[0.055199962, 0.05013141, 0.04787038, 0.01624...","[[0.011956477, 0.14930709, 0.026600504, 0.0527...","[0.3958578109741211, 0.2917083501815796, 0.309...","[11.857297, 11.392319, 15.061656, 7.1847134, 8...",3,3


In [37]:
predicted["cosine_sim"][0]

[0.42473626136779785,
 0.364050030708313,
 0.3477550148963928,
 0.3942415714263916,
 0.3710247278213501,
 0.18569022417068481,
 0.3519207835197449]

In [38]:
predicted["euclidean_dis"][0]

[14.563858, 15.262213, 17.398178, 14.272491, 13.339654, 9.336262, 15.720997]

## Accuracy

In [39]:
def accuracy(target, predicted):
    
    acc = (target==predicted).sum()/len(target)
    
    return acc

### Accuracy for  euclidean Distance

In [40]:
print(accuracy(predicted["target"], predicted["pred_idx_euc"]))

0.4


### Accuracy for Cosine Similarity

In [41]:
print(accuracy(predicted["target"], predicted["pred_idx_cos"]))

0.54


In [42]:
predicted.to_csv("train_detect_sent.csv", index=None)

In [44]:
predicted.iloc[54,:]

context          In 1882, Albert Zahm (John Zahm's brother) bui...
question         In what year did Albert Zahm begin comparing a...
answer_start                                                     3
text                                                          1882
sentences        [In 1882, Albert Zahm (John Zahm's brother) bu...
target                                                           0
sent_emb         [[0.065336294, 0.12845865, 0.13129659, 0.04857...
quest_emb        [[0.04265439, 0.09784849, 0.06059308, 0.097728...
cosine_sim       [0.25258445739746094, 0.3153684139251709, 0.31...
euclidean_dis          [12.38398, 13.868473, 14.315843, 14.349607]
pred_idx_cos                                                     0
pred_idx_euc                                                     0
Name: 54, dtype: object